In [1]:
%config InlineBackend.figure_format = 'retina'
%pylab inline
import sys
sys.path.insert(1, '../Codes/')
from geeCodes import *


import ee
#earthengine authenticate
ee.Authenticate()
ee.Initialize()

Populating the interactive namespace from numpy and matplotlib


Enter verification code: 4/1AWtgzh6Zi84y2SjD2PRTkKMcWUGDDYBx-hOp00asrIh0_xvB2cWVws8Dq8Q

Successfully saved authorization token.


In [2]:
####### Compute thresholds
dfc = gpd.read_file('../data_revision/cities/all/gdfCities.shp')
dfc = dfc.to_crs("EPSG:4326")
cities = dfc['UC_NM_MN']
years_str = []
years = np.arange(2002,2022)
for y in years:
    years_str.append(str(y))


In [3]:
def retutrnThreshold(day, scenario, geometry, datelist):
    
    if day == 'Day':
        cols1 = ['LST_Day_1km', 'QC_Day']
        cols2 = 'LST_Day_1km'
    else:
        cols1 = ['LST_Night_1km', 'QC_Night']
        cols2 = 'LST_Night_1km'
        
    
    if scenario == 1:
        if day=='Day':
            modis = ee.ImageCollection ('MODIS/006/MOD11A1').select(cols1)\
                  .map(toCelciusDay).filterBounds(geometry)\
                 .filter(ee.Filter.inList("system:time_start", datelist)).map(QC_Day_mask)

        
        else:
            modis = ee.ImageCollection ('MODIS/006/MOD11A1').select(cols1)\
                  .map(toCelciusNight).filterBounds(geometry)\
                  .filter(ee.Filter.inList("system:time_start", datelist)).map(QC_Night_mask)
        LST90 = ee.Image((modis.select(cols2)).reduce(ee.Reducer.percentile([90])))
        
        lst90 = LST90.reduceRegion(**{
          'reducer': ee.Reducer.percentile([90]),
          'geometry': geometry.getInfo(),
          'scale': 1000,
          'crs': 'EPSG:4326',
        })
        
        return lst90.getInfo()['LST_%s_1km_p90'%day]
    
    elif scenario == 2:
        if day=='Day':
            modis = ee.ImageCollection ('MODIS/006/MOD11A1').select(cols1)\
                  .map(toCelciusDay).filterBounds(geometry)\
                 .filter(ee.Filter.inList("system:time_start", datelist)).map(QC_Day_mask)

        
        else:
            modis = ee.ImageCollection ('MODIS/006/MOD11A1').select(cols1)\
                  .map(toCelciusNight).filterBounds(geometry)\
                  .filter(ee.Filter.inList("system:time_start", datelist)).map(QC_Night_mask)

        LST90 = ee.Image((modis.select(cols2)).reduce(ee.Reducer.mean()))  ### Image
        lst90 = LST90.reduceRegion(**{
          'reducer': ee.Reducer.percentile([90]),
          'geometry': geometry.getInfo(),
          'scale': 1000,
          'crs': 'EPSG:4326',
        })

        return lst90.getInfo()['LST_%s_1km_mean'%day]
    
    else:
        if day=='Day':
            modis = ee.ImageCollection ('MODIS/006/MOD11A1').select(cols1)\
              .map(toCelciusDay).filterBounds(geometry).filterDate("2002-01-01", "2023-01-01").map(QC_Day_mask)
        else:
            modis = ee.ImageCollection ('MODIS/006/MOD11A1').select(cols1)\
              .map(toCelciusNight).filterBounds(geometry).filterDate("2002-01-01", "2023-01-01").map(QC_Night_mask)
            

        LST90 = ee.Image((modis.select(cols2)).reduce(ee.Reducer.percentile([90])))
        lst90 = LST90.reduceRegion(**{
          'reducer': ee.Reducer.percentile([90]),
          'geometry': geometry.getInfo(),
          'scale': 1000,
          'crs': 'EPSG:4326',
        })
        return lst90.getInfo()['LST_%s_1km_p90'%day]

        


In [ ]:
cols = ['city', 'Tr_day3', 'Tr_night3']
dfTreshold = pd.DataFrame(columns=cols)

In [ ]:
for idx in range(0,200):
    city = cities[idx]
    geo = dfc[dfc['UC_NM_MN']==city].reset_index(drop=True)
    S = returnCityBoundary(geo)
    
    ####### GEE Geometry ###############################################
    geometry = returnGeometry(S[0])
    
    # 2. Read temperature file
    fileT = '../data_revision/cities/air_temperature/%s/temperature_new.csv'%city
    dfT = pd.read_csv(fileT)
    
    ### GEE datelist
    datelist = listHotDays(dfT, years_str)

#    lst_day1 = retutrnThreshold('Day', 1, geometry, datelist)
#    lst_night1 = retutrnThreshold('Night', 1, geometry, datelist)
#    
#    lst_day2 = retutrnThreshold('Day', 2, geometry, datelist)
#    lst_night2 = retutrnThreshold('Night', 2, geometry, datelist)
    
    lst_day3 = retutrnThreshold('Day', 3, geometry, datelist)
    lst_night3 = retutrnThreshold('Night', 3, geometry, datelist)
    
    
    data = [city, lst_day3, lst_night3]
    dfTreshold = dfTreshold.append(pd.DataFrame(columns=cols, data=[data]), ignore_index=True)
    print(idx)

In [ ]:
dfTreshold.to_csv('../data_revision/dfTreshold3.csv', index=None)